In [2]:
import requests
import json
from bs4 import BeautifulSoup as bs
from api.iterating_api import generator_over_review
from api.parsers_api import parse_GameSpot, headers, HTTP_error

In [ ]:
status

In [49]:

url_404 = "http://www.gamespot.com/pc/strategy/dominions3/review.html?sid=6163352"
data_path = "/Users/fedor/Desktop/meta_project/data/metacritic/critic_review"
logs_path = "/Users/fedor/Desktop/meta_project/data/metacritic/logs/logs_1.txt"
save_path = "/Users/fedor/Desktop/meta_project/data/metacritic/outter_critic_review/"


In [52]:
def my_predicate(d):
    return d["name"] == "GameSpot"


a = generator_over_review(data_path,
                          # game_search_list=['INSIDE'],
                          # platform_search_list=["pc"],
                          review_predicate = my_predicate,
                          add_info=False)
COUNTER = 0

print("Do you want to delete pld logs? Enter + if so")
if input() == "+":
    open(logs_path, 'w').close() #clean up logs file
    print("-log file cleaned up")

with open(logs_path, "a") as log_file:

    for rev in a:
        COUNTER += 1
        if COUNTER < 2036:
            continue
        if not rev['ref']:
            continue
        try:
            parse_GameSpot(rev['ref'],save_path = save_path + str(COUNTER)+".json", json_save = True)

        except HTTP_error as e:

            if e.status_code == 404:
                log_file.write("404 "+e.link + "\n")
            elif e.status_code == 0:
                log_file.write("000 " + e.link + "\n")
            else:
                log_file.write(str(e.status_code) + " " + e.link + "\n")

        except requests.ConnectionError or requests.HTTPError or requests.exceptions.RequestException or requests.exceptions.MissingSchema:
            # print("Inner error of Requests lib", rev['ref'])
            log_file.write("INN " + rev['ref'] + "\n")

        except ... :
            log_file.write("VBL " + rev['ref'] + "\n")
            # print("Very broken link", rev['ref'])

Do you want to delete pld logs? Enter + if so


In [55]:
# error analysis
er_dict = {}
print()
with open(logs_path, "r") as log_file:
    for line in log_file.readlines():
        er_code = line[:3]
        if er_code not in er_dict.keys():
            er_dict[er_code] = 1
        else:
            er_dict[er_code] += 1

In [56]:
er_dict #total amount of our data - 10849

{'404': 3725, '500': 742, '000': 232, 'INN': 306, '502': 13}